# LIBRARY

In [1]:
import pandas as pd
import numpy as np
import gc
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt



import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

# DATA LOAD

In [2]:
# 데이터 분할(폴더) 구분
data_splits = ["train", "test"]

# 각 데이터 유형별 폴더명, 파일 접미사, 변수 접두어 설정
data_categories = {
    "회원정보": {"folder": "1.회원정보", "suffix": "회원정보", "var_prefix": "customer"},
    "신용정보": {"folder": "2.신용정보", "suffix": "신용정보", "var_prefix": "credit"},
    "승인매출정보": {"folder": "3.승인매출정보", "suffix": "승인매출정보", "var_prefix": "sales"},
    "청구정보": {"folder": "4.청구입금정보", "suffix": "청구정보", "var_prefix": "billing"},
    "잔액정보": {"folder": "5.잔액정보", "suffix": "잔액정보", "var_prefix": "balance"},
    "채널정보": {"folder": "6.채널정보", "suffix": "채널정보", "var_prefix": "channel"},
    "마케팅정보": {"folder": "7.마케팅정보", "suffix": "마케팅정보", "var_prefix": "marketing"},
    "성과정보": {"folder": "8.성과정보", "suffix": "성과정보", "var_prefix": "performance"}
}

# 2018년 7월부터 12월까지의 월 리스트
months = ['07', '08', '09', '10', '11', '12']

for split in data_splits:
    for category, info in data_categories.items():
        folder = info["folder"]
        suffix = info["suffix"]
        var_prefix = info["var_prefix"]

        for month in months:
            # 파일명 형식: 2018{month}_{split}_{suffix}.parquet
            file_path = f"open/{split}/{folder}/2018{month}_{split}_{suffix}.parquet"
            # 변수명 형식: {var_prefix}_{split}_{month}
            variable_name = f"{var_prefix}_{split}_{month}"
            globals()[variable_name] = pd.read_parquet(file_path)
            print(f"{variable_name} is loaded from {file_path}")

gc.collect()

customer_train_07 is loaded from open/train/1.회원정보/201807_train_회원정보.parquet
customer_train_08 is loaded from open/train/1.회원정보/201808_train_회원정보.parquet
customer_train_09 is loaded from open/train/1.회원정보/201809_train_회원정보.parquet
customer_train_10 is loaded from open/train/1.회원정보/201810_train_회원정보.parquet
customer_train_11 is loaded from open/train/1.회원정보/201811_train_회원정보.parquet
customer_train_12 is loaded from open/train/1.회원정보/201812_train_회원정보.parquet
credit_train_07 is loaded from open/train/2.신용정보/201807_train_신용정보.parquet
credit_train_08 is loaded from open/train/2.신용정보/201808_train_신용정보.parquet
credit_train_09 is loaded from open/train/2.신용정보/201809_train_신용정보.parquet
credit_train_10 is loaded from open/train/2.신용정보/201810_train_신용정보.parquet
credit_train_11 is loaded from open/train/2.신용정보/201811_train_신용정보.parquet
credit_train_12 is loaded from open/train/2.신용정보/201812_train_신용정보.parquet
sales_train_07 is loaded from open/train/3.승인매출정보/201807_train_승인매출정보.parquet
sales_trai

0

In [3]:
# 데이터 유형별 설정 
info_categories = ["customer", "credit", "sales", "billing", "balance", "channel", "marketing", "performance"]

# 월 설정
months = ['07', '08', '09', '10', '11', '12']

In [4]:
#### Train ####

# 각 유형별로 월별 데이터를 합쳐서 새로운 변수에 저장
train_dfs = {}

for prefix in info_categories:
    # globals()에서 동적 변수명으로 데이터프레임들을 가져와 리스트에 저장
    df_list = [globals()[f"{prefix}_train_{month}"] for month in months]
    train_dfs[f"{prefix}_train_df"] = pd.concat(df_list, axis=0)
    gc.collect()
    print(f"{prefix}_train_df is created with shape: {train_dfs[f'{prefix}_train_df'].shape}")


customer_train_df = train_dfs["customer_train_df"]
credit_train_df   = train_dfs["credit_train_df"]
sales_train_df    = train_dfs["sales_train_df"]
billing_train_df  = train_dfs["billing_train_df"]
balance_train_df  = train_dfs["balance_train_df"]
channel_train_df  = train_dfs["channel_train_df"]
marketing_train_df= train_dfs["marketing_train_df"]
performance_train_df = train_dfs["performance_train_df"]

gc.collect()

customer_train_df is created with shape: (2400000, 78)
credit_train_df is created with shape: (2400000, 42)
sales_train_df is created with shape: (2400000, 406)
billing_train_df is created with shape: (2400000, 46)
balance_train_df is created with shape: (2400000, 82)
channel_train_df is created with shape: (2400000, 105)
marketing_train_df is created with shape: (2400000, 64)
performance_train_df is created with shape: (2400000, 49)


0

In [5]:
#### Test ####

# test 데이터에 대해 train과 동일한 방법 적용
test_dfs = {}

for prefix in info_categories:
    df_list = [globals()[f"{prefix}_test_{month}"] for month in months]
    test_dfs[f"{prefix}_test_df"] = pd.concat(df_list, axis=0)
    gc.collect()
    print(f"{prefix}_test_df is created with shape: {test_dfs[f'{prefix}_test_df'].shape}")


customer_test_df = test_dfs["customer_test_df"]
credit_test_df   = test_dfs["credit_test_df"]
sales_test_df    = test_dfs["sales_test_df"]
billing_test_df  = test_dfs["billing_test_df"]
balance_test_df  = test_dfs["balance_test_df"]
channel_test_df  = test_dfs["channel_test_df"]
marketing_test_df= test_dfs["marketing_test_df"]
performance_test_df = test_dfs["performance_test_df"]

gc.collect()

customer_test_df is created with shape: (600000, 77)
credit_test_df is created with shape: (600000, 42)
sales_test_df is created with shape: (600000, 406)
billing_test_df is created with shape: (600000, 46)
balance_test_df is created with shape: (600000, 82)
channel_test_df is created with shape: (600000, 105)
marketing_test_df is created with shape: (600000, 64)
performance_test_df is created with shape: (600000, 49)


0

In [6]:
#### Train ####

train_df = customer_train_df.merge(credit_train_df, on=['기준년월', 'ID'], how='left')
print("Step1 저장 완료: train_step1, shape:", train_df.shape)
del customer_train_df, credit_train_df
gc.collect()

# 이후 merge할 데이터프레임 이름과 단계 정보를 리스트에 저장
merge_list = [
    ("sales_train_df",    "Step2"),
    ("billing_train_df",  "Step3"),
    ("balance_train_df",  "Step4"),
    ("channel_train_df",  "Step5"),
    ("marketing_train_df","Step6"),
    ("performance_train_df", "최종")
]

# 나머지 단계 merge
for df_name, step in merge_list:
    # globals()로 동적 변수 접근하여 merge 수행
    train_df = train_df.merge(globals()[df_name], on=['기준년월', 'ID'], how='left')
    print(f"{step} 저장 완료: train_{step}, shape:", train_df.shape)
    # 사용한 변수는 메모리 해제를 위해 삭제
    del globals()[df_name]
    gc.collect()

Step1 저장 완료: train_step1, shape: (2400000, 118)
Step2 저장 완료: train_Step2, shape: (2400000, 522)
Step3 저장 완료: train_Step3, shape: (2400000, 566)
Step4 저장 완료: train_Step4, shape: (2400000, 646)
Step5 저장 완료: train_Step5, shape: (2400000, 749)
Step6 저장 완료: train_Step6, shape: (2400000, 811)
최종 저장 완료: train_최종, shape: (2400000, 858)


In [7]:
#### Test ####

test_df = customer_test_df.merge(credit_test_df, on=['기준년월', 'ID'], how='left')
print("Step1 저장 완료: test_step1, shape:", test_df.shape)
del customer_test_df, credit_test_df
gc.collect()

# 이후 merge할 데이터프레임 이름과 단계 정보를 리스트에 저장
merge_list = [
    ("sales_test_df",    "Step2"),
    ("billing_test_df",  "Step3"),
    ("balance_test_df",  "Step4"),
    ("channel_test_df",  "Step5"),
    ("marketing_test_df","Step6"),
    ("performance_test_df", "최종")
]

# 나머지 단계 merge
for df_name, step in merge_list:
    # globals()로 동적 변수 접근하여 merge 수행
    test_df = test_df.merge(globals()[df_name], on=['기준년월', 'ID'], how='left')
    print(f"{step} 저장 완료: test_{step}, shape:", test_df.shape)
    # 사용한 변수는 메모리 해제를 위해 삭제
    del globals()[df_name]
    gc.collect()

Step1 저장 완료: test_step1, shape: (600000, 117)
Step2 저장 완료: test_Step2, shape: (600000, 521)
Step3 저장 완료: test_Step3, shape: (600000, 565)
Step4 저장 완료: test_Step4, shape: (600000, 645)
Step5 저장 완료: test_Step5, shape: (600000, 748)
Step6 저장 완료: test_Step6, shape: (600000, 810)
최종 저장 완료: test_최종, shape: (600000, 857)


In [8]:
train_df.to_csv("train_df.csv",index=False)
test_df.to_csv("test_df.csv", index=False)

In [10]:
train_df['Segment'].value_counts(normalize=True) * 100 

Segment
E    80.08550
D    14.55175
C     5.31625
A     0.04050
B     0.00600
Name: proportion, dtype: float64

# null값

In [10]:
import pandas as pd

# Null 값 개수 확인
null_counts = train_df.isnull().sum()

# 전체 데이터 대비 Null 비율 계산
null_percentage = (null_counts / len(train_df)) * 100

# Null 값이 있는 컬럼만 출력
null_df = pd.DataFrame({"Null Count": null_counts, "Null Percentage": null_percentage})
null_df = null_df[null_df["Null Count"] > 0].sort_values(by="Null Percentage", ascending=False)

null_df

,Null Count,Null Percentage
연체일자_B0M,2394336,99.764000
_3순위여유업종,2377725,99.071875
_3순위납부업종,2310187,96.257792
_2순위여유업종,2302286,95.928583
_3순위교통업종,2045455,85.227292
_2순위납부업종,2033640,84.735000
최종카드론_대출일자,1988330,82.847083
_1순위여유업종,1987260,82.802500
최종카드론_신청경로코드,1958226,81.592750
최종카드론_금융상환방식코드,1958126,81.588583


In [11]:
# 3. 40% 이상 Null 값이 있는 컬럼 제거
threshold = 40  # 40% 이상 결측치는 제거
cols_to_drop = null_df[null_df["Null Percentage"] > threshold].index
print(len(cols_to_drop))
print(cols_to_drop)

train_df.drop(columns=cols_to_drop, inplace=True)

18
Index(['연체일자_B0M', '_3순위여유업종', '_3순위납부업종', '_2순위여유업종', '_3순위교통업종', '_2순위납부업종',
       '최종카드론_대출일자', '_1순위여유업종', '최종카드론_신청경로코드', '최종카드론_금융상환방식코드', 'RV신청일자',
       '_2순위교통업종', 'OS구분코드', '_3순위쇼핑업종', '_1순위납부업종', '_1순위교통업종', '_2순위쇼핑업종',
       '_3순위업종'],
      dtype='object')


In [12]:
# 수치형 변수: 중앙값 대체
num_cols = train_df.select_dtypes(include=['number']).columns
for col in num_cols:
    train_df[col].fillna(train_df[col].median(), inplace=True)

# 범주형 변수: 최빈값 대체
cat_cols = train_df.select_dtypes(include=['object']).columns
for col in cat_cols:
    train_df[col].fillna(train_df[col].mode()[0], inplace=True)

# Null 값이 모두 처리되었는지 확인
print(f"남아있는 Null 개수: {train_df.isnull().sum().sum()}")

남아있는 Null 개수: 0


# VIF

In [14]:
# 다중공선성(VIF) 계산 함수 정의
def calculate_vif(df):
    vif_data = pd.DataFrame()
    vif_data["feature"] = df.columns
    vif_data["VIF"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    return vif_data



In [ ]:
# 수치형 데이터만 선택
num_data = train_df.select_dtypes(include=[np.number])

# 6. VIF 값이 10 이상인 컬럼 제거 (반복적으로 실행)
vif_threshold = 10
while True:
    vif_df = calculate_vif(num_data)
    max_vif = vif_df["VIF"].max()

    # VIF 값이 10 이상인 경우 제거
    if max_vif > vif_threshold:
        col_to_remove = vif_df.loc[vif_df["VIF"] == max_vif, "feature"].values[0]
        print(f"Removing {col_to_remove} with VIF {max_vif}")
        num_data.drop(columns=[col_to_remove], inplace=True)
    else:
        break

print("최종 남은 컬럼 개수:", num_data.shape[1])